In [2]:
from model import BiLSTM_crf
from utils import load_data_and_labels,save_pred,transformer_x,transformer_y
from sklearn_crfsuite import metrics

x_train, y_train = load_data_and_labels('../data/train.txt')
transformer_x = transformer_x()
x_train = transformer_x.fit(x_train)
vocab_size = transformer_x.vocab_size
transformer_y = transformer_y(transformer_x.max_len)
y_train = transformer_y.to_onehot(y_train)

use_crf = True
model = BiLSTM_crf(num_labels=8, word_vocab_size = vocab_size,max_seq_len=transformer_x.max_len,use_crf = use_crf)
model = model.build()
model.summary()


# model.fit(x_train,y_train,verbose = 1,batch_size = 12,epochs= 1)
x_test, y_test = load_data_and_labels('../data/dev.txt')
x_test = transformer_x.tran(x_test)
y_test_onehot = transformer_y.to_onehot(y_test)
model.fit(x_train,y_train,validation_data = (x_test,y_test_onehot),verbose = 1,epochs= 15)

pred = model.predict(x_test)
pred = transformer_y.to_tag(pred)
print(metrics.flat_f1_score(y_test, pred,
                      average='weighted', labels=transformer_y.tags))

# group B and I results
labels = transformer_y.tags
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, pred, labels=sorted_labels, digits=3
))
save_pred(pred)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
word_input (InputLayer)      (None, 100)               0         
_________________________________________________________________
word_embedding (Embedding)   (None, 100, 100)          519800    
_________________________________________________________________
dropout (Dropout)            (None, 100, 100)          0         
_________________________________________________________________
BiLSTM (Bidirectional)       (None, 100, 120)          212160    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 100, 100)          12100     
_________________________________________________________________
crf (CRF)                    (None, 100, 8)            888       
Total params: 744,948
Trainable params: 744,948
Non-trainable params: 0
_________________________________________________________________
Trai

In [3]:
labels.remove('O')
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, pred, labels=sorted_labels, digits=3
))

             precision    recall  f1-score   support

      B-LOC      0.929     0.831     0.877      2877
      I-LOC      0.913     0.861     0.887      4394
      B-ORG      0.889     0.703     0.785      1331
      I-ORG      0.906     0.803     0.852      5670
      B-PER      0.948     0.735     0.828      1973
      I-PER      0.863     0.930     0.895      3851

avg / total      0.906     0.831     0.865     20096



只跑一个epoch，模型将所有tag都预测为‘O’，这是可以理解的，在没有得到足够的训练的情况下，这种保守的预测可以得到较高的crf.accuracy；在之后的训练中，想得到更高的accuracy就必须改变保守的策略，从而会预测出entity。

In [5]:
import os
os.mkdir('../data/15epochs')
model.save('../data/15epochs/15epochs_model.h5')

In [7]:
transformer_x.vocab_size

5198

In [8]:
transformer_x.max_len

100